In [48]:
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
import matplotlib.pyplot
# ensure the plots are inside this note

In [49]:
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputNodes, hiddenNodes, outputNodes, learningRate):
        # set number of nodes in each input, hidden, output layer
        self.inputNodes = inputNodes
        self.hiddenNodes = hiddenNodes
        self.outputNodes = outputNodes
        
        #link weights matrices, wih and woh
        #links inside the arrays are Wij
        #the link and associated weight are from node i to node j
        #W11, W22, etc
        #A matrix for the link weights between input and hidden layers; Winput_hidden of size (inputNodes x hiddenNodesnode)
        self.wih = numpy.random.normal(0.0, pow(self.hiddenNodes, -0.5), (self.hiddenNodes, self.inputNodes))
        self.who = numpy.random.normal(0.0, pow(self.outputNodes, -0.5), (self.outputNodes, self.hiddenNodes))

        # learning rate
        self.lr = learningRate
        
        # activation function is the sigmoid function
        self.activationFunction = lambda x: scipy.special.expit(x)
        
        self.display()
        
        pass

    
    # train the neural network
    def train(self, inputsList, targetsList):
        # convert inputs list to 2d array
        inputs = numpy.array(inputsList, ndmin=2).T
        targets = numpy.array(targetsList, ndmin=2).T
        
        # calculate signals into hidden layer
        hiddenInputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hiddenOutputs = self.activationFunction(hiddenInputs)
        
        # calculate signals into final output layer
        finalInputs = numpy.dot(self.who, hiddenOutputs)
        # calculate the signals emerging from final output layer
        finalOutputs = self.activationFunction(finalInputs)
        
        # output layer error is the (target - actual)
        outputErrors = targets - finalOutputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        #propogate error backwards by multiply error matrix by the transpose of the matrices of link weights, who
        hiddenErrors = numpy.dot(self.who.T, outputErrors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((outputErrors * finalOutputs * (1.0 - finalOutputs)), numpy.transpose(hiddenOutputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hiddenErrors * hiddenOutputs * (1.0 - hiddenOutputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputsList):
        # convert inputs list to 2d array
        inputs = numpy.array(inputsList, ndmin=2).T
        
        # calculate signals into hidden layer
        hiddenInputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hiddenOutputs = self.activationFunction(hiddenInputs)
        
        # calculate signals into final output layer
        finalInputs = numpy.dot(self.who, hiddenOutputs)
        # calculate the signals emerging from final output layer
        finalOutputs = self.activationFunction(finalInputs)
        
        return finalOutputs
    
    def display(self):
        print ("input Nodes:" + str(self.inputNodes))
        print("hidden Nodes:" + str(self.hiddenNodes))
        print("output Nodes:" + str(self.outputNodes))
        print ("learning factor:" + str(self.lr))
        print(self.wih)
        print(self.who)
        pass

In [50]:
# number of input, hidden and output nodes
inputNodes = 784
hiddenNodes = 200
outputNodes = 10

# learning rate
learningRate = 0.1

# create instance of neural network
nueral = neuralNetwork(inputNodes,hiddenNodes,outputNodes, learningRate)

input Nodes:784
hidden Nodes:200
output Nodes:10
learning factor:0.1
[[-0.02659391  0.03611703  0.01736512 ..., -0.01042904 -0.06036974
  -0.10880136]
 [-0.069619   -0.06226659 -0.02222478 ..., -0.07270145 -0.05236426
  -0.08295936]
 [-0.02783991 -0.05530158 -0.02674274 ...,  0.05225968  0.02780106
   0.09025519]
 ..., 
 [ 0.1250711   0.02711176  0.01356108 ..., -0.06468132  0.12874904
   0.0398103 ]
 [ 0.01625085 -0.05192099  0.02040124 ..., -0.06495166  0.07073156
  -0.03538628]
 [ 0.04434769  0.01567345 -0.09563555 ..., -0.02233426  0.02920629
  -0.13634629]]
[[-0.57980404 -0.76378118 -0.1242614  ...,  0.01311312  0.01528455
   0.29294773]
 [-0.36731315  0.46285085  0.14182318 ..., -0.01173802 -0.09675093
   0.22704595]
 [-0.54779383  0.05789203  0.08622255 ..., -0.0531833   0.04874572
   0.331809  ]
 ..., 
 [ 0.00975582 -0.31304756  0.16755064 ...,  0.0045519  -0.08650348
   0.4951445 ]
 [-0.15039309 -0.12137068  0.31221731 ..., -0.12162649  0.08458014
   0.2141723 ]
 [-0.05994016 

In [51]:
# load the mnist training data CSV file into a list
trainingDataFile = open("/insert path to /training/mnist_train_100.csv", 'r') #here
trainingDataList = trainingDataFile.readlines()
#print (training_data_list)
trainingDataFile.close()

FileNotFoundError: [Errno 2] No such file or directory: '/insert path to /training/mnist_train_100.csv'

In [ ]:
# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 5

for e in range(epochs):
    # go through all records in the training data set
    for record in trainingDataList:
        # split the record by the ',' commas
        allValues = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(allValues[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(outputNodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(allValues[0])] = 0.99
        nueral.train(inputs, targets)
        pass
    pass

In [ ]:
# load the mnist test data CSV file into a list
testDataFile = open("/insert path to /test/mnist_test.csv", 'r') #here
testDataList = testDataFile.readlines()
#print (testDataList)
testDataFile.close()

In [ ]:
# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in testDataList:
    # split the record by the ',' commas
    allValues = record.split(',')
    # correct answer is first value
    correctLabel = int(allValues[0])
    # scale and shift the inputs
    inputs = (numpy.asfarray(allValues[1:]) / 255.0 * 0.99) + 0.01
    # query the network
    outputs = nueral.query(inputs)
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    # append correct or incorrect to list
    if (label == correctLabel):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    
    pass

In [ ]:
# calculate the performance score, the fraction of correct answers
scorecardArray = numpy.asarray(scorecard)
print ("performance = ", scorecardArray.sum() / scorecardArray.size)